In [3]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate
from skimage.filters import gaussian as ski_gaussian
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift, ifft2
import numpy as np
import os
import timeit

os.chdir("../Mats")

# 2

2.1

In [ ]:
def apply_filter(image, filter):
    cop_image = np.copy(image)
    mir_image = np.pad(cop_image, np.int32(np.floor(np.max(filter.shape)/2)+1), mode="symmetric")
    for i in range(cop_image.shape[0]):
        for j in range(cop_image.shape[1]):
            cop_image [i,j] = np.sum(mir_image[i:i+filter.shape[0],j:j+filter.shape[1]]*filter)
    return cop_image

def fft_convolve(image, filter):
    return ifft2(fft2(image)*fft2(filter))

2.2

In [17]:
def waveAdd(image,a,v,w):
    img = np.copy(image)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i,j] += a*np.cos(v*i+w*j)

    return img



In [21]:
A = imread("cameraman.tif")

B = waveAdd(A,8,0.5,0.5)

plt.subplot(1,2,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,2,2)
plt.imshow(B,cmap="gray")